# Who owns Lausanne? 

## 1. Public data and owners

We obtained ftp access from the Lausanne office of cadastre. The data is a collection of ESRI shapefiles, describing roads, buildings, parcels, trees, waterbodies, and others.
Each shapefile is a collection of features, and each feature has an associated geometry (e.g. the shape of a land parcel) and associated attributes ( e.g. the commune responsible for the parcel, the parcel number).

We can explore this dataset by using GIS software that supports shapefiles. We used QGIS to explore the dataset.
We hoped to find an attribute describing the parcel owner in the parcel shapefile layer, but it wasn't there.
We had to resort to web scraping to recover this attribute.

## 1.1 Scraping owners

### 1.1.2 Download XML files
We wanted to associate each parcel in Lausanne to an owner. To do this, we divided Lausanne's surface in rectangles, and requested parcel informations for these rectangles to a service exposing the owners name.
The code for the scraping is in [`/scraping/owners/scrape_owners_to_xml.py`](/edit/scraping/owners/scrape_owners_to_xml.py).

The result of owner scraping is a set of 400 xml files, each containing parcel information for a geographical rectangle. The data are saved in the following directory: `data/raw/owners/`

For privacy reason we decided not to push any data on the online github respository.

We start exploring the raw owner xml data:

In [ ]:
!ls "data/raw/owners/" | wc -l

Each file is named after the coordinates, in the Swiss systems, of the top-left and bottom-right points bounding the scraped rectangle.

In [ ]:
!ls "data/raw/owners/" 2>/dev/null | head -10 # suppress error message by redirecting errors to null

In [ ]:
!head -n 20 "data/raw/owners/534810.4210526316_155847.0_535161.3710526315_155589.0.xml"

### 1.1.2 From XML to a single JSON
We use the `xq` and `jq` programs to extract only the features we care about from the differents XML files and save them as a list of objects in a single json file.
[`scraping/owners/multiple_xml_to_single_json.sh`](/edit/scraping/owners/multiple_xml_to_single_json.sh) is a small script leveraging the expressiveness of `jq` to efficiently concatenate the XML files into a single json, while also discarding all the attributes we have no interest in.

### 1.1.3 Remove duplicated and  clean owners JSON
The generated `proprio.json` JSON has duplicate entries, entries concerning other communes than lausanne and entries with missing owners. Furthermore, JSON is not the best format to handle tabluar data. The code in [`scraping/owners/owner_json_to_clean_csv.py`](/edit/scraping/owners/owner_json_to_clean_csv.py) clean the duplicates and tranfsorm the data in  CSV files.

### 1.1.4 Joining the owners data with the cadastre shapefiles
The result of the previous preprocessing steps is a CSV file with three columns: `commune number`, `parcel number`, and `owner name`:

In [ ]:
!head 'data/owners/proprio_lausanne.csv'

We would like to add the owner name to the attributes of the parcels shapefile that we obtained from "Office du Cadastre". To do so, we import the csv and the shapefile in QGIS, and we join this two "tables" by parcel number. The resulting geographical layer contains all the geographical features representing the parcels, and additionally the owner name for each parcel. We can now export this layer as a GeoJSON, making sure of using WGS-84 as the coordinate system,  and continue our exploration in a jupyter notebook:

## 1.2 Cadastral data - data exploration

We now have a GeoJSON, containing parcel geometries and parcel owners.

In [ ]:
#imports
import json
import pandas as pd
import matplotlib.pyplot as plt
import folium

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

In [ ]:
with open('data/owners/parcelles_wgs84.geojson') as geojson:
    geo = json.load(geojson)
# geo.features contains the list of parcelles
# example:
geo['features'][0]

We load the attributes of each geographical features in Pandas:

In [ ]:
features = [
    {'parc_num':feature['properties']['NO_PARC'],
    'owner':feature['properties']['proprio']} for feature in geo['features']
]
parcels = pd.DataFrame.from_records(features)
parcels.head()

### 1.2.1 Who is the biggest real estate owners?

We can now quickly answer questions such as who are the 30 biggest property owners in Lausanne, by using the number of parcels owned as to measure "bigness":

In [ ]:
parcels_per_owner = parcels['owner'].value_counts()
parcels_per_owner.head(30)

We can see that all thirty of biggest owners are either corporations, pension funds, or public institutions.

### 1.2.2 Unique owners

In [ ]:
owners = parcels['owner'].dropna()
print('total parcels', len(owners))
print('unique owners', len(owners.unique()))

There are 8k parcels in Lausanne. We are interestd to know how many people/societies own them. This number doesn't account for PPE. A lower bound on the owners can be estimated by discarding PPE all-together.

In [ ]:
len(owners[~owners.str.contains('PPE ')].unique())

This is a lower bound for the number of owners, although the real number is likely to be much higher since it's unlikely that all of these owners are also owners of every PPE share.

### 1.2.3 Visualizing the distribution of missing values

In [ ]:
# Compute portion of missing values
parcels['owner'].isna().mean()

22% of the parcels don't have owner information. Indeed, many parcels reperesent roads, and as such they didn't have an owner on the site we scraped. Also we didn't scrape the values for the northern part of Lausanne, which is mostly farmland and woods.
Let's try to visualize the missing values on a map:

In [ ]:
import folium
def getMap():
    return folium.Map([46.524, 6.63], tiles='cartodbpositron', zoom_start=14)

In [ ]:
map = getMap()

def style_function(feature):
    return {
        'fillColor':
            'red' if feature['properties']['proprio'] is None\
            else 'blue', #red for missing value, blue for valid
        'stroke': False
    }

geo_fol = folium.GeoJson(geo, style_function=style_function)

map.add_child(geo_fol)
map

This visualization is not very snappy or legible, but we can interpret it as follow:

- Red areas are parcels for which the proprietary is `None`. The northern parts of Lausanne were not scraped, since we didn't want to overload the scraped website and anyway they're mostly rural. So it is expected that they are red.
- Zooming into central Lausanne, we see that roads have unknown owners. This is also expected.
- Some areas are an overlap of Blue and Red, yielding purple parcels. This is because the dataset is slightly dirty, and some bigger parcels with no owner contain in their surface smaller parcels with known owners.

Having asserted that the dataset is fairly sane, we can drop the features were owner is `None`, since they will be of no use to us, and will make map drawing slower.

In [ ]:
geo_clean = geo.copy()

In [ ]:
# replace the list of features by filtering out the features having None as proprio
geo_clean['features'] =\
    [feature for feature in geo['features']
     if feature['properties']['proprio'] is not None]

geo = geo_clean
features = [
    {'parc_num':feature['properties']['NO_PARC'],
    'owner':feature['properties']['proprio']} for feature in geo['features']
]
parcels = pd.DataFrame.from_records(features)

###  1.2.4 Show parcels by owner type

The parcel owner format allows us to know the category of each owner. Societies are  by having 'AG' or 'SA' in their name. Similary for coperatives, foundations, and pension funds. We display a map colored by the owner category.

In [ ]:
def categorize(owner):
    if owner is None:
        return None
    owner = owner.lower()
    
    if owner == 'commune de lausanne' or\
        'dfire' in owner or\
        'cff' in owner:
        return 'public'
    elif 'retraites' in owner or\
        'pensions' in owner or\
        'prévoyance' in owner:
        return 'pension'
    elif 'fondation' in owner:
        return 'foundation'
    elif ' s.a.' in owner or\
        ' sa' in owner or\
        ' ag' in owner or\
        'société anonyme' in owner:
        return 'AG'
    elif 'ppe ' in owner or\
        'copropriété' in owner:
        return 'PPE'
    elif 'société coopérative' in owner:
        return 'coop'
    else:
        return 'private' #not true but let's approximate

In [ ]:
owners_categories = parcels[['owner']]
owners_categories['category'] = owners_categories['owner'].apply(categorize)
owners_categories = owners_categories.drop_duplicates().set_index('owner')

In [ ]:
map = getMap()

def style_function(feature):
    colors = {
        'coop': 'yellow',
        'AG' : 'red',
        'public' : 'green',
        'private': 'blue',
        'PPE': 'orange',
        'pension': 'purple',
        'foundation' : 'brown'
        
    }
    owner = feature['properties']['proprio']
    cat = owners_categories.loc[owner][0]
    
    return {
        'stroke':False,
        'fillColor': colors[cat]
    }

folium.GeoJson(geo_clean, style_function=style_function, tooltip=folium.GeoJsonTooltip(['proprio'])).add_to(map)
map

## 2. Rents data

### 2.1 Scraping

In order to analyse how ownership patterns influence prices, we needed to complement the owners dataset with rent prices.
Rent prices are generally not public, but we can scrape from real-estate websites current rent listings, and extract the prices from there.
We therefore proceeded to scrape [anibis.ch](https://www.anibis.ch/fr/default.aspx) and [homegate.ch](https://www.homegate.ch/fr) to extract up-to-date datapoints.
The scripts use are the following:

- for homegate: [`scraping/homegate/scrape_homegate.py`](/edit/scraping/homegate/scrape_homegate.py), a single well structure python program
- for anibis:
    1. [`scraping/anibis/anibis_scrape_listings.py`](/edit/scraping/anibis/anibis_scrape_listings.py) to download the index of results matching rents in lausanne
    2. [`scraping/anibis/anibis_parse_listings.py`](/edit/scraping/anibis/anibis_scrape_listings.py) to parse the listings index.
    3. [`scraping/anibis/anibis_scrape_offers.py`](/edit/scraping/anibis/anibis_scrape_offers.py) to download each single rent offer, given a parsed index
    4. [`scraping/anibis/anibis_parse_offers.py`](/edit/scraping/anibis/anibis_parse_offers.py) to parse the pages for each offer.

We agreed on a shared format to simplify the merging of these two data sources.

### 2.2 Removing duplicates

Most rent listings are published on several websites. When merging the two data sources, we first need to figure out which results are present in both datasets to avoid duplicate datapoints. We consider listings to be duplicates if they have the same address and the same price. The code is in [`cleaning/merge_rent_offers.py`](/edit/cleaning/merge_rent_offers.py) .

### 2.3 Mapping street addresses to coordinates
The data cleaning up to now provided us with a list of json objects, each one representing a rent offer.

In [ ]:
!head -20 data/rents/merged.json

The address is in textual form. To perform geographical queries on it, we need to convert it to coordinates. To do so, we use the cadastral layer of building addresses, provided by the Cadastral offic of Lausanne.
During merging of the anibis and homegate datasets, the addresses were standardized to use the format used by this cadastral layer.

To map an address to a coordinates couple, we iterate over all buildings in Lausanne, and check if the street name and the street number match those of our address. If there's a match, we extract the coordinates of the building from the cadastral layer.

This is done in [cleaning/address_to_coords.py](/edit/cleaning/address_to_coords.py) .

### 2.4 Visualizing the rents dataset

In [ ]:
# load the json of rent prices
rent_prices = json.load(open('data/rents/rent_prices.json'))

In [ ]:
# load the geojson featuring borders for each quartier
quartiers = json.load(open('data/quartiers.geojson'))

In [ ]:
# compute the cost per squared meter of the rent
for offer in rent_prices:
    offer['CHF/m2'] = float(offer['price'])/float(offer['surface'])

In [ ]:
# draw a map showing the location of each vacancy, and the quartiers borders
m = getMap()
folium.GeoJson(quartiers).add_to(m)
for offer in rent_prices:
    coords = offer['position']
    # Marker wants first the N coordinate and then E
    folium.Marker((coords[1], coords[0]), tooltip=offer['CHF/m2']).add_to(m)
m

We see that several points fall outside of the city borders. But we still have a significant number of avaialable datapoints. We will soon drop the outliers.

### 2.5 Mapping rent datapoints to quartiers
Each rent data-point has a pair of coordinates localizing it in space. _quartiers_ are polygons, whose perimeter is a list of coordinates. We can use the python library `shapely`, that allows us to perform geometrical queries, to find the _quartier_ for each rent offer.

In [ ]:
#import the two data structures needed
from shapely.geometry import Point, Polygon

for offer in rent_prices:
    offer['quartier'] = None
    for quartier in quartiers['features']:
        if quartier['properties']['Name'] == '90 - Zones foraines':
            #skip because we don't have owner data for zones foraines
            continue
        
        offer_pos = Point(offer['position'])
        #we extract the list of coordinates of the polygon vertices, discarding useless height
        quartier_vertices = [(east, north) for east, north, z in quartier['geometry']['coordinates'][0]]
        quartier_poly = Polygon(quartier_vertices)
        if quartier_poly.contains(offer_pos):
            offer['quartier'] = quartier['properties']['Name']

In [ ]:
'%06x' % (hash('ad') % (256**3))

In [ ]:
# let's sanity check by changing the color of the marker depending on the found category

m = getMap()
folium.GeoJson(quartiers).add_to(m)
for offer in rent_prices:
    coords = offer['position']
    
    #little hack to assign a different color to each quartier
    color = '%06x' % (hash(offer['quartier']) % (256**3))
    
    # Marker wants first the N coordinate and then E
    folium.CircleMarker(
        (coords[1], coords[0]),
        radius=5, fill_color='#'+color, weight=0, fill_opacity=1
    ).add_to(m)
m

In [ ]:
#display median price per neighborhood
rents_per_quartier = pd.DataFrame.from_dict(rent_prices)
rents_per_quartier[['CHF/m2', 'quartier']].groupby('quartier').agg('median')

## 3. Linear model describing relation of ownership and price
Next, we will try to extract further insight from the data